<h1 align="center"> Sensordatenfusion Tutorium 04 </h1>

<h3 align="center"> Messmodel und Updater </h3>


Ziel dieses Tutoriums ist es, $x_{k|k}$ und $P_{k|k}$ zu erhalten. 
Dies wird ermöglicht durch eine neue Messung $z_k$. 
Zudem benötigen wir noch die Messmatrix $H_k$ und ein Rauschen $R_k$.

$R$ = $\begin{bmatrix}
  \sigma ² & 0  \\
  0 & \sigma ²  \\
\end{bmatrix}$

Das Vorgehen in der Filterung ist nun folgendes:

(1) Berechnung der Innovation $v_{k|k-1}$ und ihrer Kovarianz $S_{k|k-1}$ 

  $v_{k|k-1}$ = $z_k$ - $H_k x_{k|k-1}$
  
  $S_{k|k-1}$ = $H_k P_{k|k-1} H^T_k$ + $R_k$
  
(2) Berechnung der Gewichtsmatrix $W_{k|k-1}$  

  $W_{k|k-1}$ = $P_{k|k-1} H^T_k S_{k|k-1}$


(3) Berechnung $x_{k|k}$ und $P_{k|k}$

  $x_{k|k}$ = $x_{k|k-1} + W_{k|k-1} v_{k|k-1}$ 
  
  $P_{k|k}$ = $P_{k|k-1} - W_{k|k-1} S_{k|k-1} W^T_{k|k-1}$

In [1]:
import scipy as sp
import numpy as np
from scipy.stats import multivariate_normal

from stonesoup.base import Property
from stonesoup.types.array import CovarianceMatrix
from stonesoup.models.base import LinearModel, GaussianModel
from stonesoup.models.measurement.base import MeasurementModel

In [2]:
import numpy as np
from scipy.stats import multivariate_normal

from stonesoup.types.array import CovarianceMatrix
from stonesoup.models.base import LinearModel, GaussianModel
from stonesoup.models.measurement.base import MeasurementModel

<h3 align="center"> Messmodell erstellen </h3>

In [3]:
class SDFMessmodell(MeasurementModel, LinearModel, GaussianModel):
    
    @property
    def ndim_meas(self):
        return 2
    
    def matrix(self, **kwargs):
        # model_matrix = np.array([["""Erste Zeile der Messmatrix"""], [""" Zweite Zeile der Messmatrix"""]])
        model_matrix = np.array([[1, 0, 0, 0], [0, 0, 1, 0]])
        return model_matrix
    
    def covar(self):
        sigma = 50
        cov = CovarianceMatrix([[np.power(sigma, 2), 0], [0, np.power(sigma, 2)]])
        return cov
    
    def rvs(self):
        # sample ziehen aus der Kovarianzmatrix
        noise = multivariate_normal.rvs(np.zeros(self.ndim_meas), self.covar(), 1)
        return noise.reshape((-1, 1))
    
    def pdf(self):
        pass
    
    
    

In [4]:
#Messmatrix H
messmodell = SDFMessmodell(4, (0, 2))

In [5]:
messmodell.matrix()

array([[1, 0, 0, 0],
       [0, 0, 1, 0]])

In [6]:
messmodell.covar()


CovarianceMatrix([[2500,    0],
                  [   0, 2500]])

<h3 align="center"> Imports für den Updater </h3>

In [7]:
from stonesoup.updater.base import Updater
from stonesoup.types.hypothesis import SingleHypothesis
from stonesoup.types.prediction import GaussianMeasurementPrediction
from stonesoup.types.update import GaussianStateUpdate

<h3 align="center"> Updater erstellen </h3>

In [8]:
class SDFUpdater(Updater):
    def get_measurement_prediction(self, state_prediction, measurement_model = None, **kwargs):
        x_pre = state_prediction
        Messmatrix = measurement_model.matrix
        messprediction = Messmatrix @ x_pre
        return messprediction

    def update(self, hypothesis, measurementmodel, **kwargs):
        
        measurement_matrix = measurementmodel.matrix()  # H
        measurement_noise_covar = measurementmodel.covar()  # R
        prediction_covar = hypothesis.prediction.covar  # P
        messprediction = self.get_measurement_prediction(hypothesis.prediction.mean, measurementmodel)

        S = measurement_matrix @ prediction_covar @ measurement_matrix.T + measurement_noise_covar  # S
        W = prediction_covar @measurement_matrix.T @ np.linalg.pinv(S)  # W
        Innovation = hypothesis.measurement.state_vector - (measurement_matrix @ hypothesis.prediction.mean)    # v
        
        x_post = hypothesis.prediction.mean + W @ Innovation    # x + W @ v
        P_post = prediction_covar - (W @ S @ W.T)  # P - ( W @ S @ W.T )


        # Augment hypothesis with measurement prediction
        hypothesis = SingleHypothesis(hypothesis.prediction,
                                      hypothesis.measurement,
                                      GaussianMeasurementPrediction(
                                          messprediction, S,
                                          hypothesis.prediction.timestamp)
                                      )

        return GaussianStateUpdate(x_post,
                                   P_post,
                                   hypothesis,
                                   hypothesis.measurement.timestamp)